# Hungry Geese Go West!

This is the simplest of all possible agents. Hungry Geese Go West!

This is mostly intended as a testcase to understand the basic game mechanics.

- Game board is 11x7 == 77 squares total (larger than an 64 bit number)
- Squares are numbered as if a 1D array counting from top-left, horizontal first, to bottom-right  
- `obs.geese[obs.index] = []` represents all the squares your goose currently occupues
  - `obs.geese[obs.index][0]`  is your head
  - `obs.geese[obs.index][-1]` is your tail
- `obs.food = []` is where the food is
- `conf.hunger_rate = 40` represents the number of turns before losing a segment
  - if you shrink to 0 segments you die
  - if you are the last goose standing, you win
- Game runs for 200 turns 
    - you will have lose a total of 5 squares to hunger by the last move
- You have 1 second to act
- Random agent dies very quickly as it often tries to move back over its own body 
- Agents must return a string from `['NORTH', 'EAST', 'SOUTH', 'WEST']`
  - Action is an enum with the valid moves, but return the `.name` string
  - Returning a number is invalid
  - Not returning anything makes you go NORTH

Hungry Geese Go West!

In [ ]:
%%writefile main.py

# BUGFIX: Kaggle Submission Environment os.getcwd() == "/kaggle/working/"
# https://www.kaggle.com/c/rock-paper-scissors/discussion/216159
import os
import glob
if os.environ.get('GFOOTBALL_DATA_DIR', ''):
    os.chdir('/kaggle_simulations/agent/')
    print("os.getcwd() ", os.getcwd())
    print("glob.glob(*)", glob.glob("*"))
    print("os.environ  ", os.environ)

    
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import random
    
# obs  {'remainingOverageTime': 60, 'index': 1, 'step': 30, 'geese': [[56], [68], [51, 52], [54]], 'food': [8, 15]}
# conf {'episodeSteps': 200, 'actTimeout': 1, 'runTimeout': 1200, 'columns': 11, 'rows': 7, 'hunger_rate': 40, 'min_food': 2, 'max_length': 99}
def go_west_agent(obs, conf):
    print("obs ", obs)
    # print("conf", conf)

    ## Action.NORTH, Action.EAST, Action.SOUTH, Action.WEST
    # return random.choice(list(Action)).name;  # random agent dies quickly
    return Action.WEST.name  # "WEST"

In [ ]:
%run main.py

# Play

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("hungry_geese", debug=True)
env.run([
    go_west_agent, 
    go_west_agent, 
    # go_west_agent, 
    # go_west_agent
])
env.render(mode="ipython", width=500, height=450)

# Evaluation

This lets us average out scores over a number of games

In [ ]:
import numpy as np
from joblib import Parallel, delayed

results = Parallel()( 
    delayed(evaluate)("hungry_geese", [
        go_west_agent, 
        go_west_agent, 
        go_west_agent, 
        go_west_agent
    ], num_episodes=1) 
for _ in range(10) )

print("results")
display(results)

print("mean", np.mean(results, axis=0).astype(np.int).flatten())
print("std ", np.std( results, axis=0).astype(np.int).flatten())
print("min ", np.min( results, axis=0).astype(np.int).flatten())
print("max ", np.max( results, axis=0).astype(np.int).flatten())

# Submission

# Submission

Multi-file submissions are a new Kaggle feature since Halite.
> Today we released support for multi-file agents. To upload an agent with multiple files, submit your work as a .tar.gz archive (the name must end in .tar.gz) with a python file at the top level called main.py that conforms to the normal agent submission requirements. The maximum upload size of 100MB applies to the full archive. For the initial release of this feature we only compile the main.py file so to import other python files you'll either have to compile/exec or importlib the relevant files. We're hoping to improve this in a future release and will update this topic as appropriate.
> - https://www.kaggle.com/c/halite/discussion/177686

In [ ]:
!GZIP=-9 tar cfz submission.tar.gz *.py